# Visualizing SuperQuadrics with Marching Cubes and Marching Tetrahedra
Matthew Lind,
University of Illinois at Urbana-Champaign

In this lab, you will implement the Marching Cubes and Marching Tetrahedra algorithms to generate isosurfaces from volumetric data.  The superquadric primitives will be employed as the volumetric data to be visualized to illustrate the differences.  Before proceeding, read chapter 2 "Marching Cubes and Variants" from the book __Isosurfaces: Geometry, Toplogy, and Algorithms__ by Rephael Wenger (CRC Press / AK Peters Publishing).  The book can be downloaded for free as a .pdf from the CRC Press website when accessed through the UIUC library.

__Marching Cubes__ is an algorithm for generating isosurfaces from volumetric data by subdividing a volume into virtual cubes and marching through the volume evaluating each cube in turn.  The results of each cube are stitched together to form a contiguous polygon mesh as the isosurface.  Marching Cubes can be easily implemented with the use of a lookup table and performs in linear time making it a practical choice for generating isosurfaces.  However the algorithm has a few weaknesses.  One weakness is data in some cubes may be ambiguous leading to uncertainty as to how the isosurface should be constructed.  Either additional computation, or conscious biased choices in the lookup table must be made to resolve ambiguities and produce a cohesive isosurface.  Reference: [Marching Cubes (Wikipedia)](https://en.wikipedia.org/wiki/Marching_cubes)

__Marching Tetrahedra__ is a variation employed by many who couldn't use Marching Cubes due to patent restrictions.  Marching Tetrahedra can be viewed as an extension of Marching Cubes as it follows the same principle but dissects the cubes into tetrahedra producing more granularity which resolves all ambiguity and produces a correct isosurface every time.  However, this comes at the expense of additional computation and significantly more triangles and vertices in the resulting isosurface.  Within the Marching tetrahedra algorithm are variants decomposing the cube into 5 tetrahedra or 6 tetrahedra.  Each has their strengths and weaknesses as we'll see later.  Reference: [Marching Tetrahedra (Wikipedia)](https://en.wikipedia.org/wiki/Marching_tetrahedra).

__Superquadrics__ are a class of primitives defined by variations of the sphere equation.  In addition to scientific visualization, superquadrics have been employed in computer graphics under the name of 'metaballs' or 'blobby particles' for density based modeling techniques favorable to haptic sculpting and simulating modeling with clay.  Metaballs are usually enhanced with features such as attractive and repulsive blending enabling metaballs to act as carving and shaping tools in the modeling process, and to represent liquids and other formless shapes.  When the number of metaballs becomes large, representation moves to point clouds employing blobby particles.   One example of superquadrics used as metaballs in commercial filmmaking is the main animated character in the 1997 movie [Flubber](https://en.wikipedia.org/wiki/Flubber_(film)).  Commerical software applications using this technique include [Z-Brush (PixelLogic)](https://pixologic.com/) and [MudBox (AutoDesk)](https://www.autodesk.com/products/mudbox/overview).   Reference: [SuperQuadrics (Wikipedia)](https://en.wikipedia.org/wiki/Superquadrics)

In [ ]:
import numpy as np
import pyvista as pv
import time

from test.test import *

### constants ###
DEBUG = False
X = 0    # convenience for accessing indices in vectors, arrays, ...
Y = 1
Z = 2

In [ ]:
#=======================================================================
# StopWatch() - Primitive stopwatch for tracing execution times during debugging/testing
#=======================================================================
class StopWatch( object ):
    def __init__( self ):
        self.start = time.perf_counter()
        self.split = self.start
        self.total = 0
        
    # Get current interval - returns time elapsed since last split
    def GetSplit( self ):
        now        = time.perf_counter()
        duration   = ( now - self.split )
        self.split = now
        return( duration )

    # Returns total time elapsed since object was initialized
    def GetTotal( self ):
        self.total = ( time.perf_counter() - self.start )
        return( self.total )

    # Re-initializes stopwatch to the current time.
    def Reset( self ):
        self.start = time.perf_counter()
        self.split = self.start
        self.total = 0

---
# Part 1: Generate a polygon mesh from data.

In this lab, the isosurface is represented as a polygon mesh because it can represent almost any surface due to it's arbitrary nature, and most algorithms can be employed without sophisticated mathematics.  However, polygon meshes are approximations of the surface they represent as all vertices are connected by linear edges. For example, it would require near infinite resolution to represent a highly curved surface as the linear edges would require many vertices to be placed close together to approximate the curvature making for an impractical choice.  The result is always a compromise between the desired output and available computing resources.

Polygon meshes can be created in PyVista using the pyvista.PolyData() class.  PolyData requires two lists:
1. Vertex Positions
2. Polygon connectivity.

Vertex positions are provided as XYZ position vectors described in the coordinate space of the object.  Each position vector is a 1D array of scalars of size 3.

The polygon connectivity is provided as a list of face descriptions.  Each face description a 1D array of __N+1__ entries, where __N__ represents the number of sides in the polygon and is the first value in the face description.  The trailing values are the indices of the vertices in the vertex positions list which form the polygon and specified in the order that respects the _right hand rule_.  The last specified index will automatically be connected by an edge to the first specified index to complete the polygon.  Face descriptions must be provided as row vectors, hence the use of np.hstack() below to format the list.  Keep that in mind if using something other than numpy to form your data.

__Right hand rule__:  Using the fingers of your right hand like a loose fist to flow (point) in the ordered direction the vertices are connected, your extended thumb will point in the direction of the surface normal.  The surface normal will always be perpendicular to the plane of the polygon and point in the direction considered the 'front' side of the polygon.  If viewing the polygon's front side, the vertices will be connected in a counter-clockwise direction.  If the order is incorrect, the polygon will be inverted or corrupt.  Exact details of how this situation is handled varies by API.

In the code block below, construct two polygon mesh cubes, each from a vertex positions list and polygon connectivity list.  For the first polygon mesh, desribe the faces as triangles.  In the second polygon mesh, describe each face as a quadrilateral polygon.  Place the first vertex at (0,0,0), the remaining vertices along the positive X, Y, and Z axes, and constrained to range [0...1].  For this exercise, assume the X and Y axes span the ground plane, and the Z axis is the vertical axis.

In [ ]:
# mesh points
vertices = np.array([
    # your code here
])

# faces - triangle mesh
facesT = np.hstack([
    # your code here
])

# faces - quadrilateral mesh
facesQ = np.hstack([  
    # your code here
])

aPolygonMeshes = [
    pv.PolyData( vertices, facesT ),
    pv.PolyData( vertices, facesQ )
]

## Display the polygon mesh

One option to display the resulting polygon mesh is to use the Pyvista.Plotter(). This method is useful when you want to see multiple polygon meshes in a single view, or when you need to decorate the view with information.

The general procedure for creating a view with Pyvista.Plotter() is as follows:

- Create a subplot.
- Add elements to the subplot (e.g. a polygon mesh)
- Specify camera parameters.
- (optional) Decorate with text.

Code for a simple plotter is provided below.

In [ ]:
#===================================================================================================
# plotMeshes() - plots polygon meshes in a view
#
# Parameters:
# polygonMeshes: (list) polygon meshes to be displayed
# cameraPosition: (3D vector) position coordinate for the camera
# cameraLookAt: (3D vector) position coordinate the camera will look at.
# titles: (list) text to display at the top of the plot
# imageRes: (2D vector) pixel resolution of the plot
# imageName: (string) name for the rendered image, including file extension.
#
# returns: None
#===================================================================================================
def plotMeshes( polygonMeshes, cameraPosition, cameraLookAt, textData, imageRes, imageName ):
    
    nbMeshes = len( polygonMeshes )
    
    plotter = pv.Plotter( shape=(1, nbMeshes ) )

    for i in range( 0, nbMeshes ):

        mesh = aPolygonMeshes[i]

        # create a new subplot to occupy part of the final frame.
        plotter.subplot(0,i)

        # set camera
        plotter.camera_position = [
            cameraPosition, # position
            cameraLookAt,   # look-at
            (0,0,1)         # up vector
        ]

        # apply text
        plotter.add_text( 
            textData[i],
            font_size=12,
            font='courier'
        )

        # add the mesh with colormap
        plotter.add_mesh( mesh, show_edges=True, scalars=mesh.points[:,Z] )#, scalar_bar_args={'title:', 'Z coordinate'}  )

        # add scalar bar
        # plotter.add_scalar_bar( "Z" )

    # display the result
    plotter.show(  window_size=imageRes, screenshot=imageName )

In [ ]:
titles = ["Cube - Triangles", "Cube - Quadrilaterals"]
textData = []

nbMeshes = len( aPolygonMeshes )

# build text captions
for i in range(0,nbMeshes):
    mesh = aPolygonMeshes[i]
    textData.append(
        titles[i] +
        '\n  Vertices: \t' + str( len( mesh.points ) ) +
        '\n  Polygons: \t' + str( mesh.n_faces )
)

# display the meshes
plotMeshes( aPolygonMeshes, [3,3,2], [0.5, 0.5, 0.5], textData, [1920,720], "cubes.png" )

 Your cubes should look like this:

![cubes](./images/polygon_mesh_cubes.png)

---
# Part 2: Implement SuperQuadric()

For this lab we'll visualize a superquadrics primitive.  Superquadrics are a class of primitives defined by a variation of the sphere equation:
## $$(\frac{|x|}{a})^r + (\frac{|y|}{b})^s + (\frac{|z|}{c})^t = R^2$$
where:
- __x,y,z__: position coordinate of a point on the surface.
- __a,b,c__: local axis scale factors of the superquadric primitive.
- __r,s,t__: exponents which determine the shape of the primitive
- __R__: Radius of the superquadric at point [x,y,z].

The primitive is based on an ellipsoid, but can become any number of other shapes by modifying the scale and exponent parameters:

![SuperQuadric shapes](./images/superquadric_shapes.png)


We must modify the equation to generate an isosurface by rearranging the equation to solve for the radius __R__, where R will be compared to our isovalue.  For example, if isovalue = 1.0, we would make the following comparison:

- __R < 1__: position is inside the primitive.
- __R = 1__: position is on surface of the primitive.
- __R > 1__: position is outside the primitive.

The position coordinate will be derived from the grid points of our scalar field.  Each point will be tested to see if it is inside or outside the primitive and recorded in the grid point as the isovalue (scalar value).

In the cell below, implement the superquadric function to read a position coordinate and return the radius at that location.  SuperQuadric() should accept multiple arguments as input:
-  __position__: (3D vector) position in 3D space to be evaluated (default = [0,0,0])
-     __scale__: (3D vector) scaling factors of the primitive's local axes (default = [1,1,1])
- __exponents__: (3D vector) Exponents used in the equation which define the primitive's shape (default = [2,2,2])

The function should return a scalar indicating the radius from the center of the superquadric primitive.

In [ ]:
#===========================================================================
# superQuadric() - given position (x,y,z), determine if it is inside our outside the primitive.
# If less than 1.0, point is inside the surface.
#
# Parameters:
#  position: (3D vector) position in 3D space to be evaluated (default = [0,0,0])
#     scale: (3D vector) scaling factors of the primitive's local axes (default = [1,1,1])
# exponents: (3D vector) Exponents used in the equation which define the primitive's shape (default = [2,2,2])
#
# Returns:
# (scalar): radius of the primitive
#===========================================================================
def superQuadric( position=[0,0,0], scale=[1,1,1], exponents=[2,2,2] ):
    
    # your code here
    
    return None

In [ ]:
## try your code here

position     = [1,2,0]
sq_scale     = [1,1,1]
sq_exponents = [2,2,2]

r = superQuadric( position, sq_scale, sq_exponents )
print( "radius: ", r )

In [ ]:
### test cases (not exhaustive) ###

test_superquadric( superQuadric )

---
# Part 3: Define the Volume

Before we can visualize anything, we must define the domain where visualization will occur.  We'll define a _scalar field_ - a three dimensional grid of aribtrary dimension whose points contain a scalar value of importance relative to the data we're visualizing.  Resolution of the field is defined by the space between the points and shaped liked cubes, known as _cells_.  

The Marching Cubes and Tetrahedra algorithms traverse the scalar field's cells gathering scalars from the nearest grid points and applying them to the corners.  The cell is then evaluated by comparing the scalars to a specified isovalue.  If the isovalue is between the scalar values of adjacent vertices, then the isosurface intersects that edge and a vertex is inserted at the intersection derived from a linear interpolation process of the participating vertex positions and scalar values.  After the cell is evaluated, the inserted vertices are connected to form polygons, then later merged with polygons of adjacent cells to form the complete isosurface.  The scalar field will be traversed in the X, Y, then Z order (with Z being the vertical axis) to correlate with the lookup tables we'll define later.

The generated isosurface will be affected by a few factors:
- __Cell size__: Distance between adjacent grid points along an axis in the scalar field, and uniform across all axes.
- __Dimensions__: Size of scalar field in each axis.  Defined by 2 positions representing the min and max corners of the bounding box.
- __IsoValue__: Scalar value at which the isosurface will be drawn.

Cell size affects the resolution of the isosurface.  Smaller sizes bring scalar field points closer together producing an isosurface with finer details, but increase computation and memory requirements significantly as more points are needed to cover a field of the same dimensions.

Dimensions of the scalar field affect how much of the isosurface is seen.  If the isosurface extends beyond the dimensions of the scalar field, it will be truncated at the field's boundary.  This can be a useful tactic to visualize a portion of a larger data set by transforming the data (or the field) so only the area of interest appears in the scalar field.

Isovalue affects the shape of the isosurface relative to the data set.  This is subjective and varies with the data set.

![Scalar Field](./images/scalar_field.png)

## Implement ScalarField and Pole classes

The Pyvista CurviLinear grid or RectiLinear grid classes would normally be used, but they are cumbersome for our purposes, so we'll devise our own classes: __ScalarField__ and __Pole__.
 
The __ScalarField__ class implements the scalar field as a modified CurviLinear grid.  Points are connected in matrix-like fashion with a position and scalar value recorded at each grid point.  The data structure storing this information is known as a _pole_.  The scalar field needs to know the dimensions of the grid in the X, Y, and Z directions, and populate the grid with a pole at each grid point.

ScalarField should be initialized with the number of _cells_ in each of the major axes, actively store the dimensions of the field, and an array or list for the poles.  The ScalarField class should include the following methods:

- __\_\_init\_\_( x, y, z )__: intializes the scalar field to the specified dimensions.
   - __x__: (int) Number of cells in the scalar field's X dimension.
   - __y__: (int) Number of cells in the scalar field's Y dimension.
   - __z__: (int) Number of cells in the scalar field's Z dimension.
   
   
- __setVertex( i, j, k, value )__: Writes a pole to the specified coordinate with value _value_.
   - __i, j, k__: (int) Index of the grid point in the scalar field's X, Y, and Z dimensions.  
     The most negative and positive corners of the field have coordinates (0,0,0) and (X,Y,Z) respectively.
   - __value__: (scalar) value to store in the pole object stored at the vertex.
   
   
- __getVertex( i, j, k )__: Returns the pole stored at the specified coordinate.
   - __i, j, k__: (int) Index of the grid point in the scalar field's X, Y, and Z dimensions.

The __Pole__ class defines a pole as a 3D position vector and a scalar value, storing each independently.  It should include the following methods:

- __\_\_init\_\_( x, y, z, value )__: initializes the x, y, z, and value properties to the specified values, or 0.
- __getValue()__: returns the scalar portion of the pole.
- __setValue( value )__: sets the scalar portion of the pole to scalar _value_.
- __getVector()__: returns the vector portion of the pole.

Feel free to add your own properties/methods as needed.

In [ ]:
#=========================================
# ScalarField()
# 
# defines 3D scalar field with values at the grid points.
#=========================================
class ScalarField( object ):
    def __init__( self, x=0, y=0, z=0 ):
        self.x = 0
        self.y = 0
        self.z = 0
        
    #-------------------------------------------------------------------------------------
    # getIndexFromLocation() - given a grid position coordinate, return it's index in the flattened array
    #-------------------------------------------------------------------------------------
    def getIndexFromLocation( self, i, j, k ):
        # your code here
        return None

    #-------------------------------------------------------------------------------------
    # getVertexFromIndex() - given a grid point index, return the pole stored at the location
    #-------------------------------------------------------------------------------------
    def getVertexFromIndex( self, index ):
        # your code here
        return None

    #-------------------------------------------------------------------------------------
    # getVertex() - returns pole stored at specified coordinate
    #-------------------------------------------------------------------------------------
    def getVertex( self, i, j, k ):
        # your code here
        return None

    #-------------------------------------------------------------------------------------
    # setVertex() - Records pole 'p' at specified coordinate (i,j,k)
    #-------------------------------------------------------------------------------------
    def setVertex( self, i, j, k, p ):
        # your code here
        return None
    
#=========================================
# Pole()
#
# associates a 3D vector with a scalar value
#=========================================
class Pole():
    def __init__( self, x=0, y=0, z=0, value=0 ):
        self.x     = x
        self.y     = y
        self.z     = z
        self.value = value

    #-------------------------------------------------------------------------------------
    # getValue() - return value portion of the pole
    #-------------------------------------------------------------------------------------
    def getValue( self ):
        # your code here
        return None

    #-------------------------------------------------------------------------------------
    # setValue() - set value portion of the pole
    #-------------------------------------------------------------------------------------
    def setValue( self, value ):
        # your code here
        return None

    #-------------------------------------------------------------------------------------
    # getVector() - return the vector portion of the pole
    #-------------------------------------------------------------------------------------
    def getVector( self ):
        # your code here
        return None
    

In [ ]:
## try your code here.

field = ScalarField( 5, 5, 5 )


In [ ]:
### test cases (not exhaustive) ###

test_scalarfield( ScalarField, Pole )

## Implement createScalarField() 

createScalarField() creates a scalar field with the ScalarField class, then calls superQuadric() at each grid point to compute and store the scalar values as poles.  Since we're only handling a single superquadric primitive centered at the origin, and rotations are not considered, we'll provide arguments to the superquadric function too.

Implement the createScalarField() function in the cell below.  It should accept the following arguments:

- __cellSize__: (scalar) Edge length of each cell in the scalar field (i.e. distance between grid points). default = 0.25.
- __bbmin__: (3D vector) position coordinate defining the most negative corner of the field (default = [-5,-5,-5]
- __bbmax__: (3D vector) position coordinate defining the most positive corner of the field (default = [5,5,5]).
- __scale__: (3D vector) Local axis scale factors for the superquadric primitive (default = [1,1,1])
- __exponents__: (3D vector) Exponents for the superquadric function (default = [2,2,2]).

The function should return the object representing the fully created scalar field, or __None__ upon failure.

In [ ]:
#===================================================================================
# createScalarField() - Creates a scalar field using superquadric() to assign 
#                       scalar values to the points as poles
#
# Parameters:
# cellsize: (scalar) dimension of a cell in the scalar field.
# bbmin: (3D vector) location of the lower corner of the scalar field
# bbmax: (3D vector) location of the upper corner of the scalar field.
# scale: (3D vector) scaling factor for the superquadric primitive
# exponents: (3D vector) exponents for the superquadric primitive
#
# Returns:
# scalar field: (object) the resulting scalar field object, or None upon error.
#===================================================================================
def createScalarField( cellSize=0.25, bbmin=[-5,-5,-5], bbmax=[5,5,5], scale=[1,1,1], exponents=[2,2,2] ):
    # your code here
    return None

In [ ]:
### try your code here ###


In [ ]:
### testing.  Set flag for test mode ###

test_mode = 0       # 0 = low resolution scalar field, 
                    # 1 = medium resolution scalar field (NOTE: may take a few minutes), 
                    # 2 = high resolution scalar field (WARNING: may take really long time)

In [ ]:
### test cases (not exhaustive) ###

test_createScalarField( createScalarField, test_mode )

### Create scalar field and SuperQuadric primitive

Creating a scalar field can be quite time consuming if the field has a high resolution.  So as a measure of practicality, we'll define a scalar field with the superquadric primitive here and reuse it in the steps below to develop the marching algorithms.

In [ ]:
cellSize     = 0.20
isovalue     = 1.0
bbmin        = [-5,-5,-5]         # min corner of bounding box
bbmax        = [5,5,5]            # max corner of bounding box
dimensions   = np.subtract( bbmax, bbmin )
sq_scale     = [4,4,4]
sq_exponents = [4,4,4]

# create the mesh
sw = StopWatch()
field = createScalarField( cellSize, bbmin, bbmax, sq_scale, sq_exponents )

print( "createScalarField(): %.6f seconds" % sw.GetTotal() )
print( " cell counts[x,y,z]:", field.dimension )

---
# Part 4: Marching Tetrahedra 6

In this section, we'll implement Marching Tetrahedra 6 as it's the simplest of the marching algorithms.

Marching Tetrahedra is an extension of Marching Cubes where each cell of the scalar field is subdivided into tetrahedra.  Tetrahedra are simpler to evaluate for intersections with the isosurface and avoid ambiguities that arise in Marching Cubes.  The algorithm comes in two versions: 5 tetrahedra and 6 tetrahedra.

One advantage to Marching Tetrahedra is there are far fewer unique arrangements of positive poles to consider as tetrahedra have fewer edges.  Marching Cubes has 256 overall cases, but only 22 are unique.  Marching Tetrahedra has 16 cases, but only 5 are unique (only 3 producing geometry), and no ambiguities to resolves because no matter how a edges are bisected, the result is always a triangle with clear orientation.

One disadvantage of the tetrahedra method is it takes longer to compute, and generates more triangles, edges, and vertices in the isosurface.  The additional geometry does not necessarily improve the detail of the isosurface.

## Table driven programming

As discussed in Wenger's book, computing intersections for tetrahedra is fairly straightforward.  Since there are a limited number of configurations of positive poles, it means we know all the possible outcomes in advance.  We gain a performance advantage by precomputing the results and inserting them into a table, then looking up the correct solution as needed at runtime.

Below are the lookup tables for Marching Tetrahedra 6 to assist the construction of the isosurface when traversing the cells.  Later in the project you will be responsible for constructing the equivalent tables for Marching Tetrahedra 5 and Marching Cubes.  The tables can certainly be changed to other data structures or formats, but the important detail is the relationships of the values in the table.  Spend time getting familiar with them.  It may not make sense until later in the project when implementing the classes.

The lookup table is divided into three primary data structures: 
- vertex indices of Cell edges.
- Tetrahedra vertex indices.
- Polygon connectivity.

__aIndicesSource6__ and __aIndicesTarget6__ describe the vertex pairs that form an edge in the cell.  It is used as a convenience to march around the cell testing for intersections with the isosurface, and for copying vertex indices between cells.  Given an edge index 'n', aIndicesSource6[n] represents the first cell vertex index of the edge, and aIndicesTarget6[n] represents the second cell vertex index of the edge.

__aTetraIndices6__ defines the tetrahedra of the cell.  Each index of this table contains the indices of the 4 cell vertices which define a tetrahedron.  The indices are always listed in ascending order.  This means they do not describe the direction polygons wind on the tetrahedra (nor is that important).

__aTriangleEdgeIndices6__ defines the order in which edges should be connected to form triangles based on the bitcode for the tetrahedra (explained later).  The row specifies the tetrahedron in the cell.  The column specifies which of the tetrahedron configurations is used to create the polygon.  The first 4 columns are 1-pole cases, the last 3 columns are 2-pole cases.

In [ ]:
### lookup tables for Marching Tetrahedra ###

# Each version of Marching tetrahedra has 4 lookup tables:
#   tetra indices:  Indices of the vertices forming each tetrahedra in the cell
#   source indices: Indices of the first vertex of each edge in the cell
#   target indices: Indices of the second vertex of each edge in the cell
# triangle indices: Indices of the *edges* containing vertices (intersections) 
#                   which form the triangle(s) of the isosurface.
#
# NOTE: the vertex and edge indices are local to the cell for consistency.
#       It is the developer's responsibilty to correlate the indices to the isosurface.
#
# Tetra indices are used each time the cell is called to evaluate
# and limit the scope of evaluation to the current tetrahedron.
#
# Each call to evalCell() generates 5 or 6 calls to evalTetra() 
# depending on which Marching Tetrahedra algorithm is in use.
# This affects the arrangement of tetrahedra within the cell and requires
# each case be handled differently.  Furthermore, in the case of 5 tetrahedra,
# the orientation of the edges within the cell changes every other cell.
# Two sets of indices must be devised for the 5 tetrahedra case.
# The two cases are specified as 5A and 5B (rotated cell).
#
# Triangle edge indices are arranged as a table where the row indicates the tetrahedron,
# and the column indicates the configuration which needs to be resolved.
# the first four are single pole cases, and the last three are two-pole cases.
# There are only three two-pole situations because some are reflections of others (i.e. same logic).
# The returned list contains the indices of the edges within the cell that contain
# intersections which must be connected to form the triangles representing the isosurface for that tetrahedron.
# The edge indices are correlated to the source and target indices tables repsectively.
#
# Example:
#   triangle edge index 14 represents the edge formed by connecting source vertex 14 and target vertex 14.
#   For Marching tetrahedra 5A = edge formed by vertices [1,6]
#   For Marching tetrahedra 5B = edge formed by vertices [2,5]
#   For Marching tetrahedra 6  = edge formed by vertices [0,7]

#--------------------------
# 5 tetrahedra
#--------------------------
# (see implementation downstream)

#--------------------------
# 6 tetrahedra
#--------------------------
#                                        1 1  1 1 1  1 1 1
#                  0 1 2 3  4 5 6 7  8 9 0 1  2 3 4  5 6 7
aIndicesSource6 = [0,1,2,3, 4,5,6,7, 0,1,2,3, 0,0,0, 6,6,6, 0] # edge start vertex
aIndicesTarget6 = [1,2,3,0, 5,6,7,4, 4,5,6,7, 2,5,7, 4,1,3, 6] # edge end vertex

aTetraIndices6 = [
    [0,4,6,7], # tetra 0
    [0,3,6,7], # tetra 1
    [0,2,3,6], # tetra 2
    [0,1,2,6], # tetra 3
    [0,1,5,6], # tetra 4
    [0,4,5,6]  # tetra 5
]

# triangle indices winding order 6 tetrahedra
aTriangleEdgeIndices6 = [
    #  1-pole and 3-poles                          2-poles
    #[ [v0],    [v1],      [v2],      [v3]         [0011, 1100]   [0101, 1010]  [0110, 1001] ]
    [ [14,18,8], [8,15,7],  [6,15,18], [14,7,6],   [18,15,7,14],  [14,6,15,8],  [18,6,7,8]   ], # tetra 0
    [ [3,18,14], [11,17,3], [18,17,6], [6,11,14],  [18,14,11,17], [3,17,6,14],  [11,6,18,3]  ], # tetra 1
    [ [18,3,12], [12,2,10], [17,2,3],  [18,10,17], [3,2,10,18],   [18,17,2,12], [3,17,10,12] ], # tetra 2
    [ [18,12,0], [0,1,16],  [10,1,12], [18,16,10], [18,12,1,16],  [18,10,1,0],  [12,10,16,0] ], # tetra 3
    [ [0,13,18], [16,9,0],  [13,9,5],  [5,16,18],  [18,16,9,13],  [18,0,9,5],   [0,16,5,13]  ], # tetra 4
    [ [18,13,8], [8,4,15],  [5,4,13],  [18,15,5],  [13,4,15,18],  [18,5,4,8],   [13,5,15,8]  ]  # tetra 5
]

## Implement the Cell class

In this section we'll implement the __Cell__ class to represent a cell in the scalar field.

A cell is a virtual cube surrounding an arbitrary location in the scalar field that isn't a grid point.  It has 8 vertices, and in the case of Marching Tetrahedra 6, has 19 edges.  The first 12 edges define the shape of the cube, and the last 7 edges represent the diagonals on the faces defining the arrangement of tetrahedra.  But you say, "wait a minute - a cube only has 6 diagonals!".  That is correct.  The 7th diagonal runs from the most negative corner to the most positive corner.  All tetrahedra in the cell share this diagonal as a common edge.

If we evaluate each cell in isolation, the resulting isosurface will be a collection of triangles casually known as a 'polygon soup'.  The generated triangles will collectively form a quilt of the isosurface, but they will not share any vertices or edges to form a contiguous surface.  Polygon soups are inefficient because they require more storage space due to duplication of features.  They also present problems when manipulated as the surface tends to split easily because the triangles are not connected, and is especially poor during rendering as numerical imprecision make the polygon mesh appear to have slivers/cracks between adjacent triangles.

The Cell class will keep track of which edges contain vertices inserted by the algorithm so triangles from adjacent cells can be stitched together to form a continguous polygon mesh.  Components of the cell must be numbered consistently to make the algorithm work.  We'll use the following vertex and edge ordering:

![Marching Tetrahedra 6](./images/tetra_6.png)

The vertices are labeled in green, the edges in yellow, and diagonal edges in red.  Of special note is edge 18 which spans opposite corners of the cell.  This edge is shared by all tetrahedra in the cell.

When the eye is placed at the most negative corner looking towards the most positive corner and centered on the last diagonal edge (edge 18), you'll see the silhouettes of the 6 tetrahedra arranged in counter-clockwise order.  Use this intuition to guide your understanding of the cell's (and tetrahedra's) role within the algorithm.  Consult the lookup tables to see how the tetrahedra are defined relative to this view.

![Marching Tetrahedra 6](./images/tetra_6_view.png)

The Cell class should include the following properties:
- __Count__: (int) Number of vertices in the cell with recorded pole values
- __values__: (list) pole values.
- __positions__: (list) pole position coordinates.
- __edgeMap__: (list) Each index represents an edge of the cell. An index's value is the vertex index inserted by the Marching algorithm, or -1 if no vertex stored on that edge (any negative value can be used, and may be helpful to use unique values for debugging purposes).

The Cell class should include the following methods:

- __\_\_init\_\_()__: initializes the cell's properties
- __addPole( pole )__: Inserts a pole at the next available corner of the cell.
- __copyEdgeMap( cellX, cellY, cellZ )__: copies vertex indices from adjacent cell(s) edges to the current cell's edges where the edges are common to both cells.
   - __cellX__: (cell) Previously traversed cell object adjacent to the current cell in the X direction.
   - __cellY__: (cell) Previously traversed cell object adjacent to the current cell in the Y direction.
   - __cellZ__: (cell) Previously traversed cell object adjacent to the current cell in the Z direction.

   __NOTE__: Depending on the location of the current cell within the scalar field, one or more adjacent cells may not be available.  copyEdgeMap() must recognize these situations and react accordingly.  A better intuition may be developed how to implement this feature when implementing the IsoMesh class in the next section.

In [ ]:
#====================================================
# Cell()
#
# cube of a scalar field recording the vertex values
# and list of which edges contain inserted vertices.
#====================================================
class Cell( object ):
    def __init__( self, count=0, values=[], positions=[] ):
        self.Count     = 0
        self.values    = []
        self.positions = []
        self.edgeMap   = [-1,-2,-3,-4, -5,-6,-7,-8, -9,-10,-11,-12, -13,-14,-15, -16,-17,-18, -19]

    # === Methods ===
    #-------------------------------------------------------------------------------------
    # clear() - resets the cell to default values
    #-------------------------------------------------------------------------------------
    def clear( self ):
        # your code here
        return None
        
    #-------------------------------------------------------------------------------------
    # addPole() - Records pole object 'p' at the next available vertex of the cell.
    #-------------------------------------------------------------------------------------
    def addPole( self, p ):
        # your code here
        return None

    #-------------------------------------------------------------------------------------
    # copyEdgeMap() - copies poles of vertices in common from specified adjacent cells to the current cell.
    #-------------------------------------------------------------------------------------
    def copyEdgeMap( self, oCelli, oCellj, oCellk ):
        # your code here
        return None

In [ ]:
# try your code here


In [ ]:
### test cases (not exhaustive) ###

test_cell( Cell, Pole )

##  Evaluate Tetrahedra

When marching through the scalar field, each cell is evaluated independently.  Intersection tests are performed along each edge of the cell by comparing the isovalue of the isosurface to the scalars stored in the poles of the cell's vertices.  If an intersection is detected, a vertex is inserted on the edge at the appropriate location.  After all edges have been tested, each tetrahedron in the cell is evaluated to construct polygon(s) by 'connecting the dots' using the appropriate inserted vertices.  In this section you'll implement `evalTetra()` to construct the polygon(s).

The evalTetra() function has the following parameters:

- __tetraIndex__: (int) Index of the tetrahedron being evaluated within the cell, in range [0...5].
- __bitCode__: (int) Each bit of the code represents one of the 8 vertices of the cell.  If a bit is nonzero, the corresponding vertex is marked as a positive pole.  For example, a bitcode of `00100101` means vertices 0, 2, and 5 are marked as positive.

The general workflow is as follows:
- Use the tetraIndex parameter to get the tetrahedron's vertices from the lookup table.
- Read the bit code to determine which of the tetrahedron's vertices are positive poles.
- Determine which edges of the tetrahedron intersect the isosurface.  An intersection occurs where the bits of adjacent vertices do not match.
- Construct a polygon description for the newly generated triangle(s).

Each triangle is a list of 4 integers where the first value is always '3' to indicate a 3-sided polygon, and the last 3 values are the edge indices of the tetrahedron containing the inserted vertices of the formed triangle specified in the order that respects the right hand rule (i.e. the surface normals of the generated triangles should point towards the positive poles).  

__Hint__: Use the lookup table to identify the correct edge indices.

Example list of 2 triangles:

    aPolygons = []
    aPolygons.append( [3,5,4,13]  )   # triangle 1
    aPolygons.append( [3,13,9,16] )   # triangle 2

In the event no triangles are generated (no intersections), return an empty list.

In [ ]:
#=======================================================
# evalTetra()
#
# Parameters:
# tetraIndex: (int) index of tetrahedron within the cell.
#    bitCode: (int) 8 digit bitcode representing positive poles of the cell.
#
# Returns:
# list of generated triangles, or empty list if no triangles generated.
#=======================================================
def evalTetra( tetraIndex, bitCode ):
    # your code here
    return None

In [ ]:
## try your code here

aPolygons = evalTetra( 0, 3 )
print( "Polygons[3]: ", aPolygons )

aPolygons = evalTetra( 4, 200 )
print( "Polygons[200]: ", aPolygons )


In [ ]:
### test cases (not exhaustive) ###

test_evalTetra( evalTetra )

## Implement IsoMesh class

This class implements the Marching Tetrahedra algorithm and contains methods to generate the isosurface.  The general workflow is:
- Initialize the isomesh with a scalar field.
- Call IsoMesh.createIsoSurface() to traverse the scalar field cell-by-cell.  Copy relevant scalar field data  into a cell object which is then passed to IsoMesh.evalCell().  Record the cell in the IsoMesh.Cells list.
- Isomesh.evalCell() computes an 8-bit code by inspecting poles of the cell.  If a pole is greater than the isovalue, a 1 is recorded for the vertex.
- Isomesh.evalCell() also computes location(s) where the isosurface intersects the cell.  When an intersection is discovered, the location of intersection is computed, a vertex is inserted into the IsoMesh.Vertices list, and the vertex index is recorded in the cell's edgeMap for the appropriate edge. The cell is then passed to isomesh.evalTetra() for each tetrahedron in the cell.
- Isomesh.evalTetra() Converts the 8-bit code to a 4-bit code for the current tetrahedron.  If intersections are detected on the edges, triangles are constructed from inserted vertices and inserted into the isomesh.Polygons list.

When a vertex is inserted into an edge of the cell, it's position is computed by linear interpolation.  If p1 and p2 are the vertex positions, and v1 and v2 are the pole values at the edge's vertices respectively, then the location of insertion is defined using a two step process:

1) compute the insertion location as a ratio from p1 to p2 by comparing the isovalue to the poles:

   $$ratio = ( isovalue - v1 )  /  ( v2 - v1 )$$
   
2) Use linear interpolation to compute the position coordinate along the edge:

   $$insertion\_position = ( ratio * p2 ) + ( (1 - ratio) * p1 )$$


The IsoMesh class should define the following properties:
- __VertexCount__: (integer) Number of vertices in the isosurface mesh.
- __Vertices__: (list) vertex positions of the isosurface (the vertex list).  Each index is a 1D array of 3 scalars (i.e. a 3D vector)
- __PolygonCount__: (integer) Number of triangles in the isosurface mesh.
- __Polygons__: (list) Each index is a 1D array of integers representing a polygon in the isosurface.  Each polygon is described as the number of sides in the polygon, followed by the indices of the vertices comprising the polygon.  The first and last vertex in the list are automatically connected to form a closed polygon.
- __CellCount__: (integer) Marker for the current index in the Cells list.
- __CellsX__: (integer) number of cells in the X direction of the scalar field.
- __CellsY__: (integer) number of cells in the Y direction of the scalar field.
- __CellsZ__: (integer) number of cells in the Z direction of the scalar field. 
- __Cells__: (list) 1D representation of a flattened 3D array of cells evaluated so far.

The IsoMesh class should define the following methods:
- __clear()__: Resets the object to default settings.
- __addCell( cell )__: appends 'cell' to the Cells list, and returns it's index to the caller.
- __getCell( index )__: returns the cell at the specified index from the Cells list.
- __addVertex( [x,y,z] )__: appends the specified position to the vertex list, and returns it's index to the caller.
   - __[x,y,z]__: (scalars) position coordinate of a vertex.
- __getVertexPosition( index )__: returns the vertex position at the specified index of the vertex list.
- __insertVertex( isovalue, p1, p2, v1, v2 )__: given an isovalue and two vertices of the cell along a common edge, compute where the isosurface intersects the edge.  Add the resulting vertex position to the vertex list, and return it's index in the vertex list to the caller.  p1, p2 are the scalar values of the edge vertices, and v1, v2 are the vertex positions.
   - __isovalue__: (scalar) value at which the isosurface is to be evaluated.
   - __p1, p2__: (scalar) pole values stored at the edge's vertices.
   - __v1, v2__: (3D vector) position coordinates of the edge's vertices.
- __addPolygon( [N, v0, v1, .., vN-1] )__: append polygon description to the polygon list, return it's index in the Polygons list to the caller.
  - __N__: (int) Number of sides of the polygon.
  - __v0...vN-1__: (int) edge indices comprising the polygon.
- __createIsoSurface( Field, isovalue )__: Traverse the scalar field and evaluate at the specified isovalue. Returns the isosurface as a polygon mesh, or __None__ if no isosurface produced.
  - __Field__: (scalar field) scalar field object to be evaluated.
  - __isovalue__: (float) isovalue of the surface to be evaluated.
- __evalCell( cell, isovalue )__: Generates the 8-bit code for the cell specifying the positive poles, determines which edges intersect with the isosurface, then inserts vertices at the intersections while recording the affected edge in the Cell's edgemap.  Calls evalTetra() once for each tetrahedron in the cell to define triangles where the isosurface intersects the tetrahedron.
  - __cell__: (cell) instance of the cell from the scalar field which is to be evaluated.
  - __isovalue__: (scalar) value at which the isosurface is evaluated.
- __evalTetra( tetraIndex, vertexIndices, bitCode, edgeMap )__: Parses the cell's bitcode to determine which vertices of the tetrahedron have positive poles, builds polygon description(s) for generated triangles which connect the inserted vertices, then adds the polygons to the IsoMesh's polygon list using the addPolygon() method.  __NOTE__: This method is slightly different from evalTetra() implemented earlier as it passes values to/from the class by-reference and calls other methods directy instead of passing data by-value.
  - __tetraIndex__: (int) index of the tetrahedron within the cell.  
  - __vertexIndices__: (list) cell's vertex indices comprising the tetrahedron.  
  - __bitcode__: (int) 8-bit code from evalCell().
  - __edgeMap__: (list) cell's edgemap which contains the vertices inserted into the cell's edges.
- __createMesh()__: Creates a polygon mesh from the Vertices and Polygons lists, then returns it to the caller as a Pyvista.PolyData object, or __None__ on failure.

Feel free to add your own properties/methods as needed.

In [ ]:
#=============================================================================
# IsoMesh() - class representing the isosurface generated by Marching Tetrahedra.
#
#=============================================================================
class IsoMesh( object ):
    def __init__( self, cellsX=0, cellsY=0, cellsZ=0 ):
        self.VertexCount   = 0
        self.Vertices      = []        
        self.PolygonCount  = 0
        self.Polygons      = []
        self.CellCount     = 0
        self.CellsX        = cellsX     # input scalar field dimensions
        self.CellsY        = cellsY
        self.CellsZ        = cellsZ
        self.CellSheetSize = ( cellsX * cellsY )
        self.Cells         = [None] * ( self.CellSheetSize + 1 )     

        self.Pow           = [1,2,4,8,16,32,64,128]        # precomputed powers of 2 for bitcode generation
        
        self._addPole          = 0
        self._copyEdgeMap      = 0
        self._evalCell         = 0
        self._evalTetra        = 0
        self._createMesh       = 0 
        self._createIsoSurface = 0

    # === Methods ===
    #=======================================================
    # clear()
    #=======================================================
    def clear( self ):
        self.VertexCount   = 0
        self.PolygonCount  = 0
        self.CellCount     = 0
        self.Vertices      = []
        self.Polygons      = []
        self.Cells         = []
        self.CellsX        = 0
        self.CellsY        = 0
        self.CellsZ        = 0
        
        self._addPole          = 0
        self._copyEdgeMap      = 0
        self._evalCell         = 0
        self._evalTetra        = 0
        self._createMesh       = 0 
        self._createIsoSurface = 0
        
    def dumpStats( self ):
        print( "    Algorithm:", 6 )
        print( "     vertices:", self.VertexCount )
        print( "     polygons:", self.PolygonCount )
        print( "    addPole(): %.6f seconds" % self._addPole )
        print( "copyEdgeMap(): %.6f seconds" % self._copyEdgeMap )
        print( "   evalCell(): %.6f seconds" % (self._evalCell - self._evalTetra) )
        print( "  evalTetra(): %.6f seconds" % self._evalTetra  )
        print( " createMesh(): %.6f seconds" % self._createMesh, )
        print( "      Total: %.6f seconds" % self._createIsoSurface )
        
    #=======================================================
    # addCell() - append cell into cell list
    #=======================================================
    def addCell( self, oCell ):
        # your code here
        return None

    #=======================================================
    # getCell() - retrieve the cell at specified index
    #=======================================================
    def getCell( self, i=0, j=0, k=0 ):
        # your code here
        return None

    #=======================================================
    # addVertex() - add vertex (x,y,z) to the vertex list
    #=======================================================
    def addVertex( self, x=0, y=0, z=0 ):
        # your code here
        return None
    
    #=======================================================
    # insertVertex() - generate a vertex at location where isovalue crosses edge p1-p2
    # then insert into the mesh.
    #=======================================================
    def insertVertex( self, isovalue, p1, p2, v1, v2 ):
        # your code here
        return None

    #=======================================================
    # addPolygon() - add polygon to the polygon list
    #=======================================================
    def addPolygon( self, n, v1, v2, v3 ):
        # your code here
        return None

    #=======================================================
    # createMesh() - given a list of vertices and list of polygons, generate a polygon mesh.
    #=======================================================
    def createMesh( self ):
        
        if ( self.VertexCount < 3 or self.PolygonCount < 4 ):
            # no mesh data
            print( "ERROR - empty mesh" )
            return( None )

        sw = StopWatch()

        try:
            mesh = pv.PolyData( np.array( self.Vertices ), np.hstack( self.Polygons ) )
        except TypeError:
            print( "ERROR: Cannot create mesh: ", TypeError )
            return( None )
        
        self._createMesh = sw.GetTotal()
        
        if DEBUG:
            # possibly useful
            for i in range( 0, self.VertexCount ):
                print( "v[" + str(i) + "]: " + str( self.Vertices[i] ) )
            for i in range( 0, self.PolygonCount ):
                print( "p[" + str(i) + "]: " + str( self.Polygons[i] ) )
            
        return( mesh  )

    #=======================================================
    # createIsoSurface() - generate an isosurface at the specified isovalue
    #=======================================================
    def createIsoSurface( self, oField, isovalue ):
        # your code here
        return None
    
    #=======================================================
    # evalCell() - compute location and orientation of polygons for the cell.
    #=======================================================
    def evalCell( self, oCell, isovalue, coord ):
        # your code here
        return None

    #=======================================================
    # evalTetra()
    #=======================================================
    def evalTetra( self, tetraIndex, aVertexIndices, bitCode, edgeMap ):
        # your code here
        return None

In [ ]:
### try your code here.  create isosurface using Marching Tetrahedra 6 ###

aPolygonMeshes = []
textData       = []

sw          = StopWatch()
isoMesh     = IsoMesh()
polygonMesh = isoMesh.createIsoSurface( field, isovalue )
duration    = sw.GetTotal()
isoMesh.dumpStats()

aPolygonMeshes.append( polygonMesh )

# build text captions
textData.append(
    "Marching Tetrahedra 6" +
    '\n cell size: \t' + str(cellSize) + 
    '\ndimensions: \t' + str(dimensions) +
    '\naxes scale: \t' + str(sq_scale) + 
    '\n exponents: \t' + str(sq_exponents) +
    '\n  Vertices: \t' + str( len( polygonMesh.points ) ) +
    '\n  Polygons: \t' + str( polygonMesh.n_faces ) +
    '\n      Time: \t' + str( duration ) + " seconds"
)

# display the meshes
plotMeshes( aPolygonMeshes, [10, 20, 8], [0,0,0], textData, [1024,1024], "mt6.png" )

In [ ]:
### test cases (not exhaustive) ###

test_isomesh( IsoMesh, createScalarField )

---
# Part 5: Marching Tetrahedra 5

Well, that was easy wasn't it? (Just kidding).  In this section you will tweak the classes to support Marching Tetrahedra 5.

Marching Tetrahedra 5 is a variant of the Marching Tetrahedra algorithm which subdivides the cell into 5 tetrahedra instead of 6.  This algorithm is more efficient as it requires less computation and generates fewer triangles.  However, to get those benefits requires a compromises which make the algorithm a bit more complicated.

If you subdivde the cell into 5 tetrahedra by drawing diagonals along each edge in an alternating pattern as you walk around the cell, you'll notice two tetrahedra can be formed at the bottom of the cell at opposite corners, and two more tetrahedra can be formed at the top of the cell also at opposite corners, but perpendicular to the bottom two tetrahedra.  This conveniently leaves a void in the middle of the cell which happens to be the 5th tetrahedron.

The challenge is the diagonals of the cell do not align with the diagonals of adjacent cells.  Fortunately, inspection reveals rotating the adjacent cells by 90 degrees on the vertical axis will resolve the problem.  This means every other cell in the scalar field must be rotated by 90 degrees to form a cohesive isosurface, which can be a bit of a headache to manage.  The cell's vertices and edges will be numbered the same as in Marching Tetrahedra 6, but the 19th edge (edge 18) no longer exists, and some of the diagonals will have an alternate orientation (numbering is not affected by change in orientation).

## Lookup tables for Marching Tetrahedra 5. 

Since there is a 90 degree rotation every other cell in the field, we must develop a lookup table for each cell orientation.  We'll discern between the two orientations of the cell with suffixes:  __5A__ indicates the cell is in default orientation, and __5B__ indicates the cell has been rotated by 90 degrees:

![Marching Tetrahedra 5](./images/tetra_5.png)

In both cases, the order of the vertices and edges will be aligned to world coordinates, not local cell coordinates.  This means vertex 0 is the most negative vertex regardless of how the cell is oriented.  For the most part, indexing for the individual cell vertices does not change, but the pairings defining the edges and tetrahedra do.  You will need to think this through.

Define the lookup tables for Marching Tetrahedra 5 in the cell below.  The simplest approach would be to duplicate the marching tetrahedra 6 tables replacing the '6' suffix with '5A' and '5B', then modify entries as needed.

- In the IndicesSource and IndicesTarget tables, each index stores the vertex index comprising the cell's edge endpoints.  
- In the TetraIndices table, define the cell vertices which comprise each of the 5 tetrahedron (in ascending order).  
- In the TriangleIndices table, define the triangles so the normals point towards the positive poles.

Generating the tables is not difficult, but debugging may be tedious.  

__TIP__: draw a diagram.

In [ ]:
### lookup tables for Marching Tetrahedra 5 ###

# Each version of Marching tetrahedra has 4 lookup tables:
#   tetra indices:  Indices of the vertices forming each tetrahedra in the cell
#   source indices: Indices of the first vertex of each edge in the cell
#   target indices: Indices of the second vertex of each edge in the cell
# triangle indices: Indices of the *edges* containing vertices (intersections) 
#                   which form the triangle(s) of the isosurface.
#
# NOTE: the vertex and edge indices are local to the cell for consistency.
#       It is the developer's responsibilty to correlate the indices to the isosurface.
#
# Tetra indices are used each time the cell is called to evaluate
# and limit the scope of evaluation to the current tetrahedron.
#
# Each call to evalCell() generates 5 or 6 calls to evalTetra() 
# depending on which Marching Tetrahedra algorithm is in use.
# This affects the arrangement of tetrahedra within the cell and requires
# each case be handled differently.  Furthermore, in the case of 5 tetrahedra,
# the orientation of the edges within the cell changes every other cell.
# Two sets of indices must be devised for the 5 tetrahedra case.
# The two cases are specified as 5A and 5B (rotated cell).
#
# Triangle edge indices are arranged as a table where the row indicates the tetrahedron,
# and the column indicates the use case which needs to be resolved.
# the first four are single pole cases, and the last three are two-pole cases.
# There are only three two-pole situations because some are reflections of others (i.e. same logic).
# The returned list contains the indices of the edges within the cell that contain
# intersections which must be connected to form the triangles representing the isosurface for that tetrahedron.
# The edge indices are correlated to the source and target indices tables repsectively.
#
# Example:
#   triangle edge index 14 represents the edge formed by connecting source vertex 14 and target vertex 14.
#   For Marching tetrahedra 5A = edge formed by vertices [1,6]
#   For Marching tetrahedra 5B = edge formed by vertices [2,5]
#   For Marching tetrahedra 6  = edge formed by vertices [0,7]


#--------------------------
# 5 tetrahedra
#--------------------------
#                                         1 1  1 1 1  1 1 1
#                   0 1 2 3  4 5 6 7  8 9 0 1  2 3 4  5 6 7
aIndicesSource5A = [0,1,2,3, 4,5,6,7, 0,1,2,3, 1,1,1, 3,3,4 ]
aIndicesTarget5A = [1,2,3,0, 5,6,7,4, 4,5,6,7, 3,4,6, 4,6,6 ]

# tetrahedra vertex indices winding order 5A
aTetraIndices5A = [
    [0,1,3,4], # tetra 0
    [1,4,5,6], # tetra 1
    [1,2,3,6], # tetra 2
    [3,4,6,7], # tetra 3
    [1,3,4,6]  # tetra 4
]

# triangle edge indices winding order 5A
aTriangleEdgeIndices5A = [
    #  1-pole and 3-poles                             # 2-poles
    #[  [v0],     [v1],       [v2],       [v3]        [0011, 1100]   [0101, 1010]  [0110, 1001] ]
    [ [0,8,3],    [13,0,12],  [12,3,15],  [15,8,13],  [8,3,12,13],   [0,8,15,12],   [13,0,3,15]   ], # tetra 0
    [ [14,9,13],  [13,4,17],  [4,9,5],    [17,5,14],  [14,9,4,17],   [13,14,5,4],   [17,13,9,5]   ], # tetra 1
    [ [1,14,12],  [2,10,1],   [12,16,2],  [14,10,16], [14,12,2,10],  [1,14,16,2],   [10,1,12,16]  ], # tetra 2
    [ [15,11,16], [7,15,17],  [17,16,6],  [6,11,7],   [11,16,17,7],  [15,11,6,17],  [7,15,16,6]   ], # tetra 3
    [ [14,13,12], [12,15,16], [15,13,17], [17,14,16], [14,13,15,16], [12,14,17,15], [16,12,13,17] ]  # tetra 4
]

#                                         1 1  1 1 1  1 1 1
#                   0 1 2 3  4 5 6 7  8 9 0 1  2 3 4  5 6 7
aIndicesSource5B = [0,1,2,3, 4,5,6,7, 0,1,2,3, 0,0,2, 0,2,5 ]
aIndicesTarget5B = [1,2,3,0, 5,6,7,4, 4,5,6,7, 2,5,5, 7,7,7 ]

# tetrahedra vertex indices winding order 5B
aTetraIndices5B = [
    [0,1,2,5], # tetra 0
    [2,5,6,7], # tetra 1
    [0,2,3,7], # tetra 2
    [0,4,5,7], # tetra 3
    [0,2,5,7]  # tetra 4
]

# triangle edge indices winding order 5B
aTriangleEdgeIndices5B = [
    #   1-pole and 3-poles                            # 2-poles
    #[  [v0],     [v1],       [v2],       [v3]        [0011, 1100]   [0101, 1010]  [0110, 1001]    ]
    [ [12,0,13],  [1,9,0],    [14,1,12],  [13,9,14],  [13,12,1,9],   [0,13,14,1],   [9,0,12,14]   ],
    [ [16,10,14], [14,5,17],  [5,10,6],   [17,6,16],  [16,10,5,17],  [14,16,6,5],   [17,14,10,6]  ],
    [ [12,15,3],  [2,16,12],  [3,11,2],   [16,11,15], [15,3,2,16],   [12,15,11,2],  [16,12,3,11]  ],
    [ [13,8,15],  [7,8,4],    [4,13,17],  [17,15,7],  [15,13,4,7],   [8,15,17,4],   [7,8,13,17]   ],
    [ [12,13,15], [16,14,12], [13,14,17], [17,16,15], [13,15,16,14], [15,12,14,17], [12,16,17,13] ]
]

## Modify the Cell class

The __Cell__ class must be updated to recognize Marching Tetrahedra 5.  Fortunately, the change is simple.  Only the copyEdgeMap() method requires modification:

__copyEdgeMap( cellX, cellY, cellZ, nbTetra )__

An argument `nbTetra` has been added to indicate how many tetrahedra the cell will be subdivided.  It can accept a value of 5 or 6.  If 5 is specified, copy the edges using the Marchng Tetrahedra 5 rules.  If 6 is specified, copy the edges using the Marching Tetrahedra 6 rules devised earlier.  If any other value is specified, return without doing any work.

If the lookup tables were defined correctly, mapping of edges between adjacent cells should be the same for both cell orientations of Marching Tetrahedra 5.

In [ ]:
### copy and paste your 'Cell' class here, then make the necessary modifications.

In [ ]:
# try your code here


## Modify the IsoMesh class

Most of the class remains unchanged, but the more important methods require modification.

- __createIsoSurface()__: Must account for cell orientation while traversing the scalar field using the Marching Tetrahedra 5 algorithm.  This affects how cells are defined, and edges copied between adjacent cells.
- __evalCell()__: Must account for tetrahedra count and cell orientation to read the correct lookup tables before calling evalTetra().
- __evalTetra()__: Must account for tetrahedra count and cell orientation to read the correct lookup tables.

Feel free to add your own changes.

In [ ]:
### copy and paste your 'IsoMesh' class here, then make the necessary modifications.

In [ ]:
### try your code here.  create isosurface using Marching Tetrahedra 5 ###

aPolygonMeshes = []
aTimes         = []
textData       = []

sw          = StopWatch()
isoMesh     = IsoMesh()
polygonMesh = isoMesh.createIsoSurface( field, isovalue, 5 )
duration    = sw.GetTotal()
isoMesh.dumpStats()

aPolygonMeshes.append( polygonMesh )

# build text captions
textData.append(
    "Marching Tetrahedra 5" +
    '\n cell size: \t' + str(cellSize) + 
    '\ndimensions: \t' + str(dimensions) +
    '\naxes scale: \t' + str(sq_scale) + 
    '\n exponents: \t' + str(sq_exponents) +
    '\n  Vertices: \t' + str( len( polygonMesh.points ) ) +
    '\n  Polygons: \t' + str( polygonMesh.n_faces ) +
    '\n      Time: \t' + str( duration ) + " seconds"
)

# display the meshes
plotMeshes( aPolygonMeshes, [10, 20, 8], [0,0,0], textData, [1024,1024], "mt5.png" )

In [ ]:
### test cases (not exhaustive) ###

test_marchingTetra5( IsoMesh, createScalarField )

---
# Part 6: Marching Cubes

Now that you have a grasp of the overall algorithm as defined by the variants, it's time to implement the original.

Marching Cubes doesn't require many modifications, and in many ways is simpler than Marching Tetrahedra.  IsoMesh.evalTetra() is no longer required, there are no diagonal edges, and Cell.copyEdgeMap() only needs to account for changes in how edges are mapped between adjacent cells in fixed orientation.

However, the lookup table is considerably larger, and ambiguity arises in resolving many of the cases.  This can be handled with computation, but it's usually more efficient and reliable to resolve the ambiguities in the lookup table.  IsoMesh.evalCell() must be modified as many cases will generate multiple triangles, as well as maneuver around the code already in place for Marching Tetrahedra when it is not required.


![Marching Cubes cell configuration](./images/cube_config.png)

## Lookup table

There are now 256 cube configurations in the lookup table vs. 16 for Marching Tetrahedra, and many cases will generate multiple triangles.

Define a table called __aCaseMap__ with 256 entries.  The first and last entry are not used as they represent the configurations of all positive poles or no positive poles resulting in no isosurface, but you must still provide a valid polygon description to prevent errors from being thrown in case something falls through the cracks.  All other cases draw at least one triangle.

Each entry in the table represents a configuration of positive poles in the cell according to the bit code.  For example, if only vertex 3 is a positive pole, the bit code would be 00001000 as expressed in binary, which evaluates to 8 in decimal.  The polygon description of the triangles drawn for that configuration should then be recorded at index 8 in the table.  The polygon description should be as defined in part 1, but with one important alteration - the vertex indices will be replaced with the cell's edge indices which contain the vertices to be connected (a double indirection).  However, in cases where multiple triangles are to be drawn, they must be appended in succession:
```
aCaseMap[
  [3,1,2,3],                  # 1 triangle with vertices [1,2,3].
  [3,1,2,3,3,4,5,6,3,7,8,9],  # 3 triangles appended one after the other with 
                                vertices [1,2,3], [4,5,6], and [7,8,9].
    
                          # NOTE: must replace vertex indices with edge indices to support indirection.
                          #       Example: vertices 0 and 1 are positive.  bitcode: 00000011, case index 3.
  [3,9,8,3,3,3,1,9],      #       <-- 2 triangles defined by vertices inserted on edges [9,8,3] and [3,1,9],
                          #       with normals facing vertices 0 and 1 while respecting the right-hand rule.
                                
]
```

![Marching Cubes polygon description](./images/cube_connections.png)

Since the cell contains no diagonals in Marching Cubes, the edge indices will range from 0 to 11 and be ordered in the same manner as used previously.  The lists must be parsed to lookup the vertex in the cell at the specified edge index, then insert it into the isosurface description (IsoMesh.Vertices, IsoMesh.Polygons) so the polygon mesh can be constructed.  In other words, same as in Marching Tetrahedra, but you may be reading  multiple triangles instead of a single triangle each time a lookup is performed.

As with Marching Tetrahedra, the polygons should be defined so the normals point towards the positive poles.

In addition to aCaseMap, define the __aIndicesSource0__ and __aIndicesTarget0__ lists to define the vertex pairings which define the cell's edges.

__TIP__: Consult Wenger's diagrams for the possible cube configurations.

__TIP__: Although the table can be generated manually, doing so is quite tedious and prone to silly mistakes.  Consider generating the table programmatically as demonstrated in the accompanying lab, __Marching_Cubes_Table\_\_lab.ipynb__.  The output can be directly copy/pasted into the cell below.

In [ ]:
aIndicesSource0 = [0,1,2,3, 4,5,6,7, 0,1,2,3]   # edge connection order in the cell
aIndicesTarget0 = [1,2,3,0, 5,6,7,4, 4,5,6,7]

### lookup table for Marching Cubes ###

# each index of the array contains polygon connectivity information.
# The first value is N, the number of sides in the polygon.
# The trailing values are the indices of the edges within the cube
# and the order they must be connected to form a polygon.
# It is the developer's responsibility to map the edge indices
# to vertex indices of the vertices on those edges so meshing can take place.
#
# NOTE: some cells contain multiple polygons and must be parsed from the returned array index.

aCaseMap = [
    [3,0,1,2],    # <-- case 0: not used

    #
    #
    # your code here (case 1 thru case 254)
    #
    #
    
    [3,4,5,6]     # <-- case 255: not used
]

---
# Part 7: Update classes

## Modify the Cell class

Once again, the Cell class must be updated to account for Marching Cubes.  Copy and paste your Cell class into the cell below, then make the following modifications:

- Implement support for Marching Cubes for cases where `nbTetra` is equal to 0.

In [ ]:
### copy and paste your 'Cell' class here, then make the necessary modifications.

In [ ]:
# try your code here


## Modify the IsoMesh class

Copy and paste your Isomesh class into the cell below, then make the following modifications:

- __createIsoSurface()__: When nbTetra = 0, use Marching Cubes. 
- __evalCell()__: When nbTetra = 0, use Marching Cubes lookup tables, and bypass evalTetra().  Must read and parse the triangle indices returned by the lookup tables for cases where multiple triangles are drawn in the cell.

In [ ]:
### copy and paste your 'IsoMesh' class here, then make the necessary modifications.

In [ ]:
### try your code here.  create isosurface using Marching Cubes ###

aPolygonMeshes = []
textData       = []

# create the mesh
sw          = StopWatch()
isoMesh     = IsoMesh()
polygonMesh = isoMesh.createIsoSurface( field, isovalue, 0 )
duration    = sw.GetTotal()
isoMesh.dumpStats()

aPolygonMeshes.append( polygonMesh )

# build text captions
textData.append(
    "Marching Cubes" +
    '\n cell size: \t' + str(cellSize) + 
    '\ndimensions: \t' + str(dimensions) +
    '\naxes scale: \t' + str(sq_scale) + 
    '\n exponents: \t' + str(sq_exponents) +
    '\n  Vertices: \t' + str( len( polygonMesh.points ) ) +
    '\n  Polygons: \t' + str( polygonMesh.n_faces ) +
    '\n      Time: \t' + str( duration ) + " seconds"
)

# display the meshes
plotMeshes( aPolygonMeshes, [10, 20, 8], [0,0,0], textData, [1024,1024], "mc.png" )

---
# Part 8: Define the SuperQuadric shape and create the IsoSurface

Now it's time to play.  Define the superquadric parameters, create the scalar field, then generate the isosurface using the 3 Marching algorithms we implemented.  Store each polygon mesh separately so we may display and compare them in a plot.

The default scalar field should span from [-5,-5,-5] to [5,5,5] with cell size of 0.25 units, but you are free to define any size you like.  Keep in mind execution time is proportional to the number of cells in the scalar field.  So pay attention to the cell size in relation to the scalar field dimensions.  If you're not careful, you can easily exceed system memory leading to extreme performance degradation or crash.

Some helpful values for the superquadric's exponents parameter:
- __(0...1)__: Concave edge.  Forms a star-like shape when applied to all axes.
- __1.0__: Linear edge.  Forms an octahedron when applied to all axes.
- __(1...2)__: Convex edge.  Forms an ellipsoid when applied to all axes.
- __2.0__: Sphere (ellipsoid).  Forms a sphere when applied to all axes.
- __(2...inf)__: Cuboid with rounded corners.  Higher exponents creates sharper corners.

![SuperQuadric shapes](./images/superquadric_shapes.png)

In [ ]:
titles         = ["Marching Cubes", "Marching Tetrahedra 5", "Marching Tetrahedra 6"]
aMethods       = [0,5,6]    # algorithms (0=Marching cubes, 5=Marching Tetrahedra 5, 6=Marching tetrahedra 6)
aPolygonMeshes = []

# plot parameters
cameraPosition = [10,25,7]
cameraLookAt   = [0,0,1]
imageRes       = [1920,720]

# superquadric parameters
sq_scale     = [4,4,4]          # scaling factor of local axes of the superqudric primitive
sq_exponents = [4,4,4]          # exponents defning the primitive's shape (see above)

# scalar field parameters
cellSize   = 0.20               # spacing between adjacent grid points along an axis
bbmin      = [-5,-5,-5]         # min corner of bounding box
bbmax      = [5,5,5]            # max corner of bounding box
dimensions = np.subtract( bbmax, bbmin )
isovalue   = 1.0

sw = StopWatch()

# generate the scalar field
field = createScalarField( cellSize, bbmin, bbmax, sq_scale, sq_exponents )
print( "create scalar field: %.6f seconds" % sw.GetSplit() )

# create an isomesh instance
isoMesh = IsoMesh( field.dimension[X] - 1, field.dimension[Y] - 1, field.dimension[Z] - 1 )

# create the isosurfaces
for i in range(0,3):
    
    print( "----------------------" )
    # reset for next isosurface
    isoMesh.clear()
    isoMesh.TetraCount = aMethods[i]
    
    sw.GetSplit()
    
    # create isosurface at isovalue
    polygonMesh = isoMesh.createIsoSurface( field, isovalue )
    if polygonMesh:
        # record the isosurface mesh
        aPolygonMeshes.append( polygonMesh )
        
    # record total time for this mesh
    duration = sw.GetSplit()
    print( "isoMesh[",i,"]", duration, " seconds" )

    textData.append(
        titles[i] +
        '\n cell size: \t' + str(cellSize) + 
        '\ndimensions: \t' + str(dimensions) +
        '\naxes scale: \t' + str(sq_scale) + 
        '\n exponents: \t' + str(sq_exponents) +
        '\n  Vertices: \t' + str( len( polygonMesh.points ) ) +
        '\n  Polygons: \t' + str( polygonMesh.n_faces ) +
        '\n      Time: \t' + str( duration ) + " seconds"
    )

# display the meshes
plotMeshes( aPolygonMeshes, cameraPosition, cameraLookAt, textData, imageRes, "final_results.png" )

---
# Part 9: Analysis
As we can see from the collected data, Marching Cubes performs better than either of the Marching Tetrahedra algorithms producing isosufaces with fewer polygons and vertices in less time.  The Marching Cubes mesh also look more aesthetically pleasing.

__TBD__

---
# Part 10: Improvements
    
Congratulations, you have completed the lab!  Be proud of your accomplishment, but be aware you have only completed a basic implementation of the Marching algorithms.  Adequate as a learning exercise, but not ideal for practice.  Consider the following:

1. The Marching Cubes algorithm produces triangles in the resulting polygon mesh. If we could merge triangles to form quadrilateral polygons in cases where the triangles are expected to be coplanar, the vertex and polygon counts could be significantly reduced saving valuable computational resources and lead to a much more pleasing result.  How much reduction can be achieved?  How would you implement this feature?  


2. The cells are accumulated as the scalar field is traversed, but we never backtrack more than an adjacent cell neighbor.  This means we store cells much longer than necessary leading to excessive memory consumption, and potentially slowing execution.  How would you fix that?


3. Our algorithm works in a classic single-threaded model.  Could we accelerate the algorithm using parallel computing?  Why or why not?


4. The Marching Tetrahedra algorithms avoid ambiguity, but introduce an excessive number of triangles that do not necessarily improve the shape of the resulting isosurface.  Why do the extra triangles not improve the isosurface?  How can this be improved?


5. Inspection reveals most of the time is spent copying values onto cell vertices whether it be from the scalar field or from adjacent cells.  How would you modify the algorithm to be more efficient in this regard?


6. What other improvements can you think of?